<a href="https://colab.research.google.com/github/jneey2000/ML/blob/main/8_introduction_to_convnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
keras.__version__

'2.7.0'

# 5.1 - Introduction to convnets


[CNN 참고자료](https://www.freecodecamp.org/news/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050/?gi=f79d86b111cb)

First, let's take a practical look at **a very simple convnet example**.  
- We will use our **convnet to classify MNIST digits**, a task that you've already been through in Chapter 2,

 <U>using a densely-connected network (**Basic DNN, DFNN**)  
(our test accuracy then was 97.8%)</U>.     
- Even though our convnet will be very basic, its
accuracy will still blow out of the water that of the densely-connected model from Chapter 2.  
(**기본적인 컨브넷(CNN)이더라도 2장의 완전 연결된 모델의 성능을 훨씬 앞지를 것임**)

- The 6 lines of code below show you what a basic convnet looks like.   
(다음 6줄 코드는 기본적인 컨브넷의 모습임)  
- <font color="blue">**It's a stack of `Conv2D` and `MaxPooling2D` layers.**</font>  
We'll see in a
minute what they do concretely.  
- Importantly, a convnet takes as <font color="blue">**input tensors of shape `(image_height, image_width, image_channels)` (not including the batch dimension)**</font>.   
- In our case, we will configure our convnet to process inputs of size `(28, 28, 1)`, which is the format of MNIST images.  
We do this via passing the argument `input_shape=(28, 28, 1)` to our first layer.

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Let's display the architecture of our convnet so far:

참고) Output shape: $\frac{(N-F+2P)}{S}+1$
- N: 입력 이미지의 크기
- F: 필터의 크기
- P: padding
- S: stride

파라미터 수 계산
- Conv2D 파라미터 수 = (필터 높이 x 필터 너비 x 입력 채널 수) x 필터 개수 + 필터 개수
예: (3 x 3 x 1) x 32 + 32 = 320

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-traina

- You can see above that <font color="blue">the output of every `Conv2D` and `MaxPooling2D` layer is **a 3D tensor of shape `(height, width, channels)`**</font>.  
(`Conv2D`와 `MaxPooling2D` 층의 출력은 `(height, width, channels)` 크기의 3D 텐서임).  
- **The width and height dimensions tend to shrink as we go deeper in the network.**  
(Width 와 height 차원 <font color="blue">**(spatial dimension)은 네트워크가 깊어질수록 작아지는 경향이 있음**</font>)  
- The number of channels is controlled by the first argument passed to the `Conv2D` layers (e.g. 32 or 64).  
(채널의 수는 `Conv2D`층에 전달된 첫 번째 매개변수에 의해 조절됨 (32개 또는 64개))

- The next step would be to feed our last output tensor (of shape `(3, 3, 64)`) into a densely-connected classifier network like those you are already familiar with: a stack of `Dense` layers.   
(다음 단계에서의 마지막 층의 출력 텐서(크기:`(3, 3, 64)`)를 완전 연결 네트워크에 주입함  
이 네크워크는 이미 익숙하게 보았던 `Dense` 층을 쌓은 분류기임)

- These classifiers process vectors, which are 1D, whereas our current output is a 3D tensor.   
(이 dense layer기반의 분류기는 1D  벡터를 처리하는 데, 이전 층의 출력이 3D 텐서임.)   
- So first, we will **have to flatten our 3D outputs to 1D**, and then add a few `Dense` layers on top  
(그래서 먼저 3D 출력을 1D 텐서로 펼쳐야 함. 그 다음에 몇 개의 Dense 층을 추가함):

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

We are going to do 10-way classification, so we use <U>a final layer with 10 outputs and a softmax activation</U>.  
Now here's what our network
looks like:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

- As you can see, our `(3, 3, 64)` outputs were flattened into vectors of shape `(576,)`, before going through two `Dense` layers.  
(`(3, 3, 64)` 출력이 `(576,)`크기의 벡터로 펼쳐진 후 `Dense` 층으로 주입됨)

Now, let's train our convnet on the MNIST digits.  
We will reuse a lot of the code we have already covered in the MNIST example from Chapter
2.

In [ ]:
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 55s 58ms/step - loss: 0.1699 - accuracy: 0.9472
Epoch 2/5
938/938 [==============================] - 55s 58ms/step - loss: 0.0455 - accuracy: 0.9858
Epoch 3/5
938/938 [==============================] - 54s 57ms/step - loss: 0.0320 - accuracy: 0.9898
Epoch 4/5
938/938 [==============================] - 53s 57ms/step - loss: 0.0246 - accuracy: 0.9924
Epoch 5/5
938/938 [==============================] - 53s 56ms/step - loss: 0.0184 - accuracy: 0.9941


Let's evaluate the model on the test data:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 11ms/step - loss: 0.0262 - accuracy: 0.9921


In [ ]:
test_acc

0.9921000003814697

While our densely-connected network from Chapter 2 had a test accuracy of 97.8%,  
our basic convnet has a test accuracy of 99.3%:  
we decreased our error rate by 68% (relative). Not bad!  
(2장의 완전 연결 네트워크는 97.8%의 테스트 정확도를 얻은 반면에  
<font color="blue">**기본적인 CNN은 99.2%의 테스트 정확도를 얻음.**</font>  
에러율이 상대적으로 64%나 줌)

But why does this simple convnet work so well, compared to a densely connected model?  
(완전 연결되 모델보다 왜 간단한 컨브넷이 더 잘 작동할까?)   
To answer this, let’s dive into **what the Conv2D and MaxPooling2D layers do**.  
(이에 답하기 위해, Conv2D 와 MaxPooling2D 층이 어떤 일을 하는 지 살펴보겠음)

## 5.1.1. The convolution operation

The fundamental difference between a densely connected layer and a convolution layer is this  
(<font color="blue">**Dense layer와 Conv layer의 근본적인 차이점**</font>):

* **Dense layers learn <span style="color:blue">global patterns</span> in their input feature space (for example, for a MNIST digit, patterns involving all pixels)**   
(<font color="blue">**Dense 층은 입력 공간에 있는 전역 패턴(예를 들어 MNIST 숫자 이미지에서는 모든 픽셀에 걸친 패턴)을 학습함**</font>)
* **Convolution layers learn <span style="color:blue">local patterns</span>** (see figure 5.1): in the case of images, patterns found in small 2D windows of the inputs. In the previous example, these windows were all 3 × 3.  
(<font color="blue">**합성곱 층은 지역 패턴을 학습함.**</font>  
이미지일 경우 작은 2D 윈도우로 입력에서 패턴을 찾음.  
앞의 예에서 이 윈도우는 모두 3x3크기임.)

**아래 그림과 같이 이미지는 에지(edge), 질감(texture) 등 지역 패턴으로 분해될 수 있음.**

<div>
<img src="https://drive.google.com/uc?export=view&id=1jGRJ8-qRbTrP1oDNmw5icfnJjUFgVu80" width="600"/>
</div>

이 핵심 특징은 CNN에 두가지 흥미로운 성질을 제공함.  
- 학습된 패턴은 <font color="blue">**위치 불변성 (translation invariant)**</font>을 가짐.  
  - CNN의 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면  
  다른 곳(예를 들어 왼쪽 위 모서리)에서도 이 패턴을 인식할 수 있음.  
  - 완전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야 함.  
  - 이런 성질은 CNN이 이미지를 효율적으로 처리하게 만들어 줌  
  (근본적으로 우리가 보는 세상은 위치 이동으로 인해 다르게 인식되지 않음)  
  - 적은 수의 훈련 샘플을 사용해서 일반화 능력을 가진 표현을 학습할 수 있음.): 파라미터 수 축소  
- CNN은 <font color="blue">**패턴의 공간적 계층 구조**</font>를 학습할 수 있음.  
  - 첫번째 합성곱 층이 에지 같은 작은 지역 패턴을 학습함.  
  - 두번째 합성곱 층은 첫번째 층의 특성으로 구성된 더 큰 패턴을 학습하는 식임.  
  - 이런 방식을 사용하여 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있음.  
  (근본적으로 우리가 보는 세상은 공간적 계층 구조를 가짐)

<div>
<img src="https://drive.google.com/uc?export=view&id=17lQdFzytJGdJ7-TUHRaBlToMPTLMKMwg" width="300"/>
</div>

출처: https://theaisummer.com/receptive-field/

<div>
<img src="https://drive.google.com/uc?export=view&id=10mchX5yCvYmCtK9h0XV0edp_hsaqG0z0" width="800"/>
</div>

- 우릭가 보는 세상은 시각적 구성 요소들의 공간적인 계층 구조로 구성되어 있음  
- 아주 좁은 지역의 에지들이 연결되어 눈이나 귀 같은 국부적인 구성 요소를 만들고 이들이 모여서 "고양이"처럼 고수준의 개념을 만듦.

<div>
<img src="https://drive.google.com/uc?export=view&id=1QCavFSdWe5U-le7pfBX9amMciH013J3F" width="700"/>
</div>

출처: https://www.kaggle.com/aakashns/advanced-transfer-learning-starter-notebook


- **Convolutions (합성곱 연산)은 feature map이라고 부르는 <font color="blue">3D 텐서**</font>에 적용됨  
- 이 텐서는 2개의 공간 축(높이와 너비)와 깊이 축(채널 축)으로 구성됨  
- RGB 이미지는 3개의 컬러 채널 (빨간색(Red), 녹색(Green), 파란색(Blue)을 가지므로 깊이 축의 차원이 3이 됨.  
- MNIST 숫자처럼 흑백 이미지는 깊이 축의 차원이 1(회색톤)임.  
- 합성곱 연산은 입력 feature map에서 **작은 패치(patch)들을 추출**하고  
 **이런 모든 패치에 같은 변환을 적용 (weigth sharing)**하여 **출력 특성 맵(output feature map)**을 만듦.  
- **Ouput feature map**도 역시 높이와 너비를 가진 **3D 텐서**임.
- 출력 텐서의 깊이는 층의 매개변수로 결정되기 때문에 상황에 따라 다름  
- 이렇게 되면 깊이 축의 채널은 더 이상 RGB 입력처럼 특정 컬러를 의미하지 않음.   
- 그 대신 일종의 필터를 의미함.  
 **필터는 입력 데이터의 어떤 특성을 인코딩함.**  
예를 들어, 고수준으로 보면 하나의 필터가 '입력에 얼굴이 있는 지'를 인코딩할 수 있음.


- MNIST 예제에서는 첫 번째 합성곱 층이 (28,28,1) 크기의 feature map을 입력으로 받아  
(26, 26, 32) 크기의 feature map을 출력함.  
- 즉, 입력에 대해 32개의 필터를 적용함.  
- 32개의 출력 채널 각각은 26x26 크기의 배열 값을 가짐.  
- 이 값은 입력에 대한 필터의 응답 맵(response map)임.

<div>
<img src="https://drive.google.com/uc?export=view&id=1hmQgAZB4fvwx-DUkj3vyQvBTFtJlkVvK" width="500"/>
</div>

<U>**Feature map (특성 맵)이란 말이 의미**</U>하는 것은 다음과 같음:   
- Depth 축에 있는 <font color="blue">**각 차원은 하나의 특성**</font>임.
- 2D 텐서 output[:, :, n]은 입력에 대한 이 필터 응답을 나타내는 **2D 공간상의 map**임.


**Conv layer를 정의하는 핵심적인 파라미터 2가지**:  
- Filter size (입력으로부터 추출할 패치의 크기): 3x3이나 1x1을 많이 사용함.  
- Filter의 수 (feature map의 출력 깊이): 앞에 예제에서는 32로 시작해서 64로 끝남.
- 케라스의 Conv2D 층에서 이 파라미터는 Conv2D(output_depth, (window_height, window_width)처럼 첫 번째와 두 번째 매개변수로 전달됨.

<div>
<img src="https://drive.google.com/uc?export=view&id=1fGPs_61RCsVI06BfcQcq68xP1R1QObfq" width="1000"/>
</div>


<div>
<img src="https://drive.google.com/uc?export=view&id=1PSuha5POPQHA057ykq8N6UZD9q1qZan4" width="800"/>
</div>

### Understanding border effects and padding (경계문제와 패팅 이해하기)  
- 5x5 크기의 feature map을 생각해보자 (총 25개의 타일이 있다고 생각)  
- 3x3 크기인 윈도우의 중앙을 맞출 수 있는 타일은 3x3 격자를 형성하는 9개 뿐임.(그림 5-5참조)  
- 따라서 output feature map은 3x3 크기가 됨.
- **크기가 조금 줄어 듦**


<div>
<img src="https://drive.google.com/uc?export=view&id=1pwNOKw8ZI2aEHsEJ9p0Ut-RERIP0ehP6" width="600"/>
</div>


- Zero padding을 사용하면 입력과 동일한 output feature map을 얻을 수 있음:  
  - Filter size: 3, Stride: 1, zero padding:1
  - Filter size: 5, Stride: 1, zero padding:2  
- zero padding은 입력 feature map 가장자리에 적절한 개수의 행과 열을 추가함



<div>
<img src="https://drive.google.com/uc?export=view&id=1cea9BSHKo6dqhKCG3xZiZkM7rex4EvOY" width="800"/>
</div>

<div>
<img src="https://drive.google.com/uc?export=view&id=1AwZaOr0UsAHvBybbBueoRBhU-rnygb8T" width="800"/>
</div>

Reference : [Convolution operation](https://www.freecodecamp.org/news/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050/?gi=f79d86b111cb)

**Stride:**  
- 출력 feature map 사이즈에 영향을 줌
- 정보 손실을 막기위해 주로 stride size는 1을 사용함.  
- 케라스에서는 stride size 1 이 default 사이즈임.
- stride 2를 사용하면 feature map의 width와 height가 2의 배수로 다운샘플링됨


##5.1.2 Max-pooling 연산  
-예제에서 feature map의 크기가 MaxPooling2D 층마다 절반으로 줄어듦  
- 예를 들어 첫 번째 MaxPooling2D 층 이전에 특성 맵의 크기는 26x26이었는 데 max-pooling 연산으로 13x13으로 줄어듦  
- 스트라이드 합성곱과 매우 비슷하게 강제적으로 feature map을 다운샘플링함.
- Max-pooling은 input feature map에서 윈도우에 맞는 패치를 추출하고 각 채널별로 최댓값을 출력함.  
- 합성곱과 개념적으로 비슷하지만 추출한 패치에 학습된 선형 변환( convolution kernel)을 적용하는 대신  
 **하드코딩된 최댓값 추출 연산을 사용**함.
- Max-pooling은 보통 2x2 윈도우와 stride 2를 사용하여 feature map을 절반 크기로 다운샘플링함.

왜 이런 식으로 feature map을 다운샘플링할까요?  
왜 max-pooling 층을 빼고 큰 feature map을 계속 유지하지 않을 까요?  
이런 방식을 한번 테스트해 보겠음  
합성곱으로만 이우어진 모델은 다음과 같음.

In [ ]:
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation='relu',
                      input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))

<div>
<img src="https://drive.google.com/uc?export=view&id=137BgwMLEGcvw4qDLS5V6gfFwnoQzmxQA" width="800"/>
</div>

In [ ]:
model_no_max_pool.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 22, 22, 64)        36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Maxpooling이 없다면 생기는 문제는?

<div>
<img src="https://drive.google.com/uc?export=view&id=14XeFN7FAliBUpHUgGx9FsK2blX-aUhzL" width="1000"/>
</div>

<font color="blue">**다운샘플링을 하는 이유:**</font>
- **처리할 특성 맵의 <U>가중치 개수</U>를 줄이기 위해서**
- **<U>Computational costs</U>를 줄이기 위해서**
- **연속적인 합성곱 층이 원본 입력에서 <U>커버되는 영역</U> 측면에서  
<U>점점 커진 윈도우</U>를 통해 바라보도록 만들어 필터의 <U>공간적인 계층 구조</U>를 구성함**

- Pooling layer 뿐만 아니라 Convolutional layer에 stride를 통해서도 다운샘플링을 할 수 있음. Stride를 2이상 사용.  
- Average pooling: 입력 패치의 채널별 평균값을 계산
- Max pooling이 다른 방법들보다 더 잘 작동하는 편임.
- 그 이유는 특성이 특성 맵의 각 타일에서 어떤 패턴이나 개념의 존재 여부를 인코딩하는 경향이 있기 때문임 (그래서 특성의 지도(맵)임)  
- 따라서, 특성의 평균값보다 여러 특성 중 최댓값을 사용하는 것이 더 유용함.